<a href="https://colab.research.google.com/github/P-Brundha/info/blob/main/23BIT012Datapreprocessingchallenge_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark kagglehub

In [ ]:
import kagglehub
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, when, count, isnan
from pyspark.ml.feature import VectorAssembler, StandardScaler



# Download latest version
path = kagglehub.dataset_download("carrie1/ecommerce-data")

print("Path to dataset files:", path)


100%|██████████| 7.20M/7.20M [00:00<00:00, 70.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/carrie1/ecommerce-data/versions/1


In [ ]:
spark = SparkSession.builder \
    .appName("DataPreprocessingChallenge") \
    .getOrCreate()

print("✅ Spark session created successfully!")


✅ Spark session created successfully!


In [ ]:
# Read CSV file
file_path = f"{path}/data.csv"  # Kaggle dataset file name
data = spark.read.csv(file_path, header=True, inferSchema=True)

print("✅ Dataset loaded successfully.")
data.printSchema()
data.show(5)


✅ Dataset loaded successfully.
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|  

In [ ]:
# Count missing values
missing_counts = data.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in data.columns])
print("🔍 Missing value count per column:")
missing_counts.show()

# Fill missing numeric columns with mean
numeric_cols = [c for c, t in data.dtypes if t in ['double', 'int']]
for column in numeric_cols:
    mean_value = data.select(mean(col(column))).collect()[0][0]
    data = data.fillna({column: mean_value})

# Fill missing categorical columns (if any)
categorical_cols = [c for c, t in data.dtypes if t == 'string']
for column in categorical_cols:
    data = data.fillna({column: 'Unknown'})

print("✅ Missing values handled.")


🔍 Missing value count per column:
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+

✅ Missing values handled.


In [ ]:
for column in numeric_cols:
    data = data.withColumn(column, col(column).cast("double"))

data.printSchema()
print("✅ Data types standardized.")


root
 |-- InvoiceNo: string (nullable = false)
 |-- StockCode: string (nullable = false)
 |-- Description: string (nullable = false)
 |-- Quantity: double (nullable = true)
 |-- InvoiceDate: string (nullable = false)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = false)

✅ Data types standardized.


In [ ]:
before = data.count()
data = data.dropDuplicates()
after = data.count()

print(f"🧹 Removed {before - after} duplicate rows.")


🧹 Removed 5268 duplicate rows.


In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Select only numeric columns
numeric_cols = ['Quantity', 'UnitPrice']

assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol="features"
)

# Assemble numeric columns into feature vector
assembled = assembler.transform(data)

# Apply StandardScaler
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(assembled)
scaled_data = scaler_model.transform(assembled)

print("✅ Data normalization completed.")
scaled_data.select("scaled_features").show(5, truncate=False)


✅ Data normalization completed.
+-------------------------------------------+
|scaled_features                            |
+-------------------------------------------+
|[-0.01651999476641918,-0.034789131080195]  |
|[-0.0393374840991838,-0.030675306390441394]|
|[-0.0393374840991838,-0.02450456935581098] |
|[-0.0393374840991838,-0.030675306390441394]|
|[-0.03477398623263087,-0.04095986811482541]|
+-------------------------------------------+
only showing top 5 rows



In [ ]:
# Drop complex vector columns before saving
clean_data = scaled_data.drop("features", "scaled_features")

# Define output path
output_path = "/content/cleaned_creditcard_data.csv"

# Save as CSV
clean_data.write.csv(output_path, header=True, mode="overwrite")

print(f"✅ Cleaned dataset saved successfully at: {output_path}")


✅ Cleaned dataset saved successfully at: /content/cleaned_creditcard_data.csv


In [ ]:
print("📊 Data Preprocessing Summary")
print("- Missing values handled (numeric: mean, categorical: 'Unknown')")
print("- Data types standardized to double")
print("- Duplicates removed")
print("- Features normalized using StandardScaler")
print("- Engineered features: Transaction_Hour, Amount_Category")
print("✅ Dataset ready for downstream analytics or ML tasks.")


📊 Data Preprocessing Summary
- Missing values handled (numeric: mean, categorical: 'Unknown')
- Data types standardized to double
- Duplicates removed
- Features normalized using StandardScaler
- Engineered features: Transaction_Hour, Amount_Category
✅ Dataset ready for downstream analytics or ML tasks.
